# 사전 설정 (colab)
- 해당 문서는 ViT 모델을 이용해 openCV로 동영상 프레임을 읽은 후,
- 감정분석을 수행한 결과를 확인하기 위한 코드파일입니다

In [ ]:
# Google Drive를 Colab 환경에 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 작업 디렉토리를 헬스케어 해커톤 모델 폴더로 변경
%cd /content/drive/MyDrive/02.git/healthcare-hackathon/model

/content/drive/MyDrive/02.git/healthcare-hackathon/model


In [ ]:
# 모듈 검색 경로에 fer2013-finetuning 디렉토리 추가
# FER2013 데이터셋 파인튜닝 관련 모듈들을 import하기 위함
import sys
sys.path.append('./fer2013-finetuning')

In [ ]:
# 필수 라이브러리 설치
# transformers: Hugging Face의 트랜스포머 모델들을 사용하기 위한 라이브러리
# FFmpeg: 비디오/오디오 처리를 위한 도구(코덱처리)
!pip install -U -q transformers
!pip install -q FFmpeg
# !pip install -q mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 131.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:

# 감정 분석 모델 관련 라이브러리 및 모듈 import
import torch  # PyTorch 딥러닝 프레임워크
from models.classifier import create_emotion_classifier  # 감정 분류 모델 생성 함수
from utils.dataset import create_dataloaders  # 데이터로더 생성 유틸리티
from utils.trainer import EmotionTrainer  # 감정 모델 훈련 클래스
from configs.config import Config  # 설정 파일

In [ ]:
from transformers import pipeline

# Hugging Face에서 사전 학습된 감정 인식 모델을 사용하여 이미지 분류 파이프라인 생성
# ViT(Vision Transformer) 기반의 얼굴 표정 인식 모델 사용
pipe = pipeline("image-classification", model="mo-thecreator/vit-Facial-Expression-Recognition")

# 테스트용 이미지(앵무새 이미지)로 감정 분석 수행
result_cls = pipe("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/hub/parrots.png")
result_cls

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda:0


[{'label': 'fear', 'score': 0.47674405574798584},
 {'label': 'sad', 'score': 0.21958568692207336},
 {'label': 'neutral', 'score': 0.12055999040603638},
 {'label': 'anger', 'score': 0.07729041576385498},
 {'label': 'happy', 'score': 0.054361917078495026}]

In [ ]:
# 감정 분석 결과 출력
print(result_cls)

# 상위 2개 감정 결과만 출력
for i, result in enumerate(result_cls):
  if i == 2:  # 2번째까지만 출력 (0, 1번 인덱스)
    break
  print(i)  # 순위 출력
  print(result['label'])  # 감정 라벨 출력


[{'label': 'fear', 'score': 0.47674405574798584}, {'label': 'sad', 'score': 0.21958568692207336}, {'label': 'neutral', 'score': 0.12055999040603638}, {'label': 'anger', 'score': 0.07729041576385498}, {'label': 'happy', 'score': 0.054361917078495026}]
0
fear
1
sad


- test video download link: https://drive.google.com/file/d/1PTK0s1d8wh4D-346cD3APsF9p0dMQMhl/view?usp=sharing

In [ ]:
# Google Drive에서 테스트 비디오 파일 다운로드
file_id = '1PTK0s1d8wh4D-346cD3APsF9p0dMQMhl'  # Google Drive 파일 ID
!gdown {file_id} -O /content/test_vid.mp4  # gdown을 사용하여 비디오 다운로드

Downloading...
From: https://drive.google.com/uc?id=1PTK0s1d8wh4D-346cD3APsF9p0dMQMhl
To: /content/test_vid.mp4
100% 4.44M/4.44M [00:00<00:00, 32.3MB/s]


In [ ]:
# OpenCV를 사용한 비디오 파일 정보 추출
import cv2
test_vid = '/content/test_vid.mp4'  # 테스트 비디오 파일 경로
cap = cv2.VideoCapture(test_vid)  # 비디오 캡처 객체 생성

# 비디오 속성 정보 가져오기
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))   # 비디오 너비
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # 비디오 높이
total_frame = cap.get(cv2.CAP_PROP_FRAME_COUNT)  # 총 프레임 수

# 비디오 정보 출력
print('width, height:',width, height)
print('total_frame:',total_frame)

width, height: 480 854
total_frame: 1668.0


In [ ]:
# 비디오 출력 설정 (사용되지 않음)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # MP4 비디오 코덱 설정
out = cv2.VideoWriter('output.mp4', fourcc, 30.0, (width, height))  # 비디오 라이터 객체 생성

In [ ]:
# PIL(Python Imaging Library) - 이미지 처리를 위한 라이브러리
from PIL import Image

In [ ]:
# 비디오에서 프레임별 감정 분석 수행
cap = cv2.VideoCapture(test_vid)  # 비디오 캡처 객체 재생성
print(cap.isOpened())  # 비디오 파일이 정상적으로 열렸는지 확인

frame_count = 0  # 프레임 카운터 초기화
emo_list = []    # 감정 분석 결과를 저장할 리스트

# 비디오의 모든 프레임을 순회하며 감정 분석 수행
while cap.isOpened():
  ret, frame = cap.read()  # 다음 프레임 읽기

  # 프레임 읽기 실패 시 종료
  if not ret:
    if emo_list == []:
      print('breaked')  # 감정 데이터가 없으면 중단됨
    else:
      print('ended')    # 정상 종료
      break

  # 500프레임마다 감정 분석 수행
  if frame_count % 500 == 0:
    # OpenCV BGR 이미지를 PIL RGB 이미지로 변환
    transform_img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # 감정 분석 수행
    result_cls = pipe(transform_img)

    # 상위 2개 감정 결과를 리스트에 추가
    for i, result in enumerate(result_cls):
      if i == 2:
        break
      emo_list.append(result['label'])

  frame_count += 1  # 프레임 카운터 증가

cap.release()  # 비디오 캡처 객체 해제
print(emo_list)  # 감정 분석 결과 리스트 출력




True
ended
['neutral', 'sad', 'neutral', 'happy', 'neutral', 'happy', 'sad', 'neutral']


In [ ]:
# 감정별 카운트를 저장할 딕셔너리 초기화
emo_map = {
    'angry' : 0,     # 화남
    'disgust' : 0,   # 혐오
    'fear' : 0,      # 두려움
    'happy' : 0,     # 기쁨
    'neutral' : 0,   # 중립
    'sad' : 0,       # 슬픔
    'surprise' : 0   # 놀람
}

# 테스트: angry 감정의 초기값 확인
emo_map['angry']

# angry 감정의 등장 횟수를 카운트하여 업데이트
emo_map['angry'] = emo_map['angry'] + emo_list.count('angry')
emo_map  # 업데이트된 감정 맵 확인

{'angry': 0,
 'disgust': 0,
 'fear': 0,
 'happy': 0,
 'neutral': 0,
 'sad': 0,
 'surprise': 0}

In [ ]:
# 모든 감정에 대해 카운트를 업데이트하고 이모지와 함께 결과 출력
for emo, count in emo_map.items():
  # 각 감정의 등장 횟수를 emo_list에서 카운트하여 업데이트
  emo_map[emo] = emo_map[emo] + emo_list.count(emo)

  # 각 감정에 해당하는 이모지 설정
  if emo == 'angry':
    face = '😡'      # 화남
  if emo == 'disgust':
    face = '🤢'      # 혐오
  if emo == 'fear':
    face = '😨'      # 두려움
  if emo == 'happy':
    face = '😄'      # 기쁨
  if emo == 'neutral':
    face = '😐'      # 중립
  if emo == 'sad':
    face = '😭'      # 슬픔
  if emo == 'surprise':
    face = '😮'      # 놀람

  # 감정별 통계 결과를 이모지와 함께 출력
  print(f'{face}감정: {emo}, ✅등장 횟수: {emo_map[emo]}\n')

😡감정: angry, ✅등장 횟수: 0

🤢감정: disgust, ✅등장 횟수: 0

😨감정: fear, ✅등장 횟수: 0

😄감정: happy, ✅등장 횟수: 2

😐감정: neutral, ✅등장 횟수: 4

😭감정: sad, ✅등장 횟수: 2

😮감정: surprise, ✅등장 횟수: 0



In [ ]:
# Jupyter 노트북에서 비디오 파일을 임베드하여 재생
from IPython.display import Video, display
display(Video('/content/test_vid.mp4', embed=True, width=width, height=height))

Output hidden; open in https://colab.research.google.com to view.